# Check out the latest Keras via TF 1.8

The first hidden layer in the network must define the number of inputs to expect, e.g. the shape of the input layer. Input must be three-dimensional, comprised of samples, time steps, and features in that order.
- Samples. These are the rows in your data. One sample may be one sequence.
- Time steps. These are the past observations for a feature, such as lag variables.
- Features. These are columns in your data.

The choice of activation function is most important for the output layer as it will define the format that predictions will take. For example, below are some common predictive modeling problem types and the structure and standard activation function that you can use in the output layer:
- Regression: Linear activation function, or linear, and the number of neurons matching the number of outputs. This is the default activation function used for neurons in the Dense layer.
- Binary Classification (2 class): Logistic activation function, or sigmoid, and one neuron the output layer.
- Multiclass Classification (> 2 class): Softmax activation function, or softmax, and one output neuron per class value, assuming a one hot encoded output pattern.
